## The algorithm below uses fuzzy search and K-nearest neighbours with the available dataset to identify the distance between the user point of origin and the distance of places of interest and then recommends the best possible points as per the threshold count

### Currently the api to integrate with the available threshold count of the recommended place is under development. This recommendation algo to be run on the data once the available , withing limit threshold values of people allowed are provided to it . 

### The dataset this model uses is minimal and is under assumption that these are the places available for the user


In [3]:
import pandas as pd
df = pd.read_excel('/Users/abhi/workspace/COC/Sarathi_Data.xlsx')

In [4]:
poi = df[['POI_ID','POI_NAME','POI_TYPE']]

In [5]:
data = df[[ 'POI_ID','POI_TYPE','POI_TYPE_ID', 'Area','POI_NAME', 'Address',
       'Calculated distance metrs', 'Threshold Count']]

In [6]:
from scipy.sparse import csr_matrix
import sklearn

In [7]:
similar_poi_pivot = data.pivot(index='POI_ID', columns='POI_TYPE_ID', values='Calculated distance metrs').fillna(0)

In [8]:
poi_to_idx = {poi: i for i , poi in enumerate(list(poi.set_index('POI_ID').loc[similar_poi_pivot.index].POI_NAME))}

In [9]:
poi_matrix = csr_matrix(similar_poi_pivot.values)

## Use of KNN model to identify nearest matrix elements

In [10]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(poi_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [11]:
from fuzzywuzzy import fuzz

/Applications/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
def fuzzy_matching(mapper, best_poi, verbose=True):
    
    match_tuple = []
    
    for key, value in mapper.items():
        ratio = fuzz.ratio(key.lower(), best_poi.lower())
        if ratio >= 50:
            match_tuple.append((key, value, ratio))
   
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('No such location!')
        return
    if verbose:
        print('Found possible matches in database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]

### Recommendation method to identify the points of interest based on the input of the user , for eg: Ganpati Temple

In [15]:
def make_recommendation(model_knn, data, mapper, best_poi, n_recommendations):
   
    model_knn.fit(data)
   
    print('Your input location:', best_poi)
    idx = fuzzy_matching(mapper, best_poi, verbose=True)
    
    print('Recommendation system will give Results soon.. ')

    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
 
    reverse_mapper = {v: k for k, v in mapper.items()}
   
    print('Recommendations for {}:'.format(best_poi))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}'.format(i+1, reverse_mapper[idx], dist))

In [18]:
## The input from the user is captured under "my_poi(my_place of interest) which will then spin out recommendation"
my_poi = 'Ganapati Temple'
make_recommendation(
    model_knn=model_knn,
    data=poi_matrix,
    best_poi=my_poi,
    mapper=poi_to_idx,
    n_recommendations=3)

Your input location: Ganapati Temple
Found possible matches in database: ['Ganapati temple', 'Anjaneya Swamy temple']

Recommendation system will give Results soon.. 
Recommendations for Ganapati Temple:
1: St Mary's Church
2: Anjaneya Swamy temple
3: Ganapati temple
